In [ ]:
import os
import base64
import ujson as json
import pandas as pd
import boto3
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()

In [ ]:
def extract_description(x):
    description = []
    extract = x[0]['attachment']['data']
    myString = base64.b64decode(extract)
    test_list=myString.decode("unicode_escape").replace('<br>', '').replace('<b>', '').replace('</b>', '').splitlines()
    description = list(test_list)
    return description
    
def extract_patient_id(x):
    x = x['reference']
    x = x.split("/")[-1]
    x = x.split("}")[0]
    return x

def extract_id(x):
    x = x['id']
    return x

In [ ]:
aws_acct_id=boto3.client('sts').get_caller_identity().get('Account')
your_bucket_export=f'hl-synthea-export-{aws_acct_id}'

In [ ]:
s3_resource = boto3.resource('s3')

s3_client = boto3.client('s3')
bucket = s3_resource.Bucket(name=your_bucket_export)

for obj in bucket.objects.all():
    if 'DocumentReference-0.ndjson' in obj.key:
        key_path=obj.key

In [ ]:
s3_resource.Bucket(your_bucket_export).download_file(key_path, 'DocumentReference-0.ndjson')

In [ ]:
records = map(json.loads, open('/home/ec2-user/SageMaker/DocumentReference-0.ndjson'))
df = pd.DataFrame.from_records(records)

In [ ]:
df.head(1)

In [ ]:
df['description'] = df['content'].apply(extract_description)
df['patient_id'] = df['subject'].apply(extract_patient_id)
df['unique_id'] = df['patient_id']+'___'+df['id']+'.txt'

In [ ]:
df_test_v1 = df[['unique_id', 'description']]
df_test_v1.head()

In [ ]:
your_bucket = f'hl-synthea-source-{aws_acct_id}'

In [ ]:
for i, row in df.iterrows():
    if i > len(df):
        break
    else:
        f = open(row['unique_id'], 'w')
        l1=map(lambda x:x+'\n', row['description'])
        f.writelines(l1)
        f.close()
        boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('source/', f.name)).upload_file(f.name)
        os.remove(f'/home/ec2-user/SageMaker/{f.name}')        
        i+=1

In [ ]:
pathtosave=os.path.abspath(os.getcwd())

for eachRow in range(0, len(df)):
    column_names = ['Score', 'Type', 'Text', 'File']
    dftestCol = pd.DataFrame(columns = column_names)
    test=df['extension'][eachRow][0]['extension']
    for diffEnt in range(0, len(test)):
        setEnt=test[diffEnt]['extension'][2:]
        for overall in range(0, len(setEnt)):
            dict_pairs = setEnt[overall].items()
            ld = list(dict_pairs) 
            if "http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity" == ld[0][1]:
                parseSecondLayer=ld[1][1]
                tempDict = {}
                for i in range(0, len(parseSecondLayer)):
                    tempDict.update({'File': 'source/'+df['unique_id'][eachRow]})
                    if 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-text' in list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Text': list(parseSecondLayer[i].values())[1]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-score' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Score': list(parseSecondLayer[i].values())[1]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-begin-offset' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Type': 'ICD10Text'})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-ConceptList' == list(parseSecondLayer[i].values())[0]:
                        additional_extension=list(parseSecondLayer[i].values())[1]
                        for j in range(0, len(additional_extension)):
                            for k in range(0, len(additional_extension[j]['extension'])):
                                if 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-Concept-Code' == list(additional_extension[0]['extension'][k].values())[0]:
                                    Code = list(additional_extension[j]['extension'][k].values())[1]
                                elif 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-Concept-Description' == list(additional_extension[0]['extension'][k].values())[0]:
                                    DescriptionText = list(additional_extension[j]['extension'][k].values())[1]
                                elif 'http://healthlake.amazonaws.com/aws-cm/infer-icd10/aws-cm-icd10-entity-Concept-Score' == list(additional_extension[0]['extension'][k].values())[0]:
                                    Score = list(additional_extension[j]['extension'][k].values())[1]
                            dftestCol = dftestCol.append(pd.DataFrame({'Score': Score, 'Type': 'ICD10Code', 'Text': Code, 'File': 'source/'+df['unique_id'][eachRow]}, index=[0]), ignore_index=True)
                            dftestCol = dftestCol.append(pd.DataFrame({'Score': Score, 'Type': 'ICD10description', 'Text': DescriptionText, 'File': 'source/'+df['unique_id'][eachRow]}, index=[0]), ignore_index=True)
                        c_maxes = dftestCol.groupby(['Type', 'Text']).Score.transform(max)
                        dftestCol = dftestCol.loc[dftestCol.Score == c_maxes]
                dftestCol = dftestCol.append(pd.DataFrame(tempDict, index=[0]), ignore_index=True)
                c_maxes = dftestCol.groupby(['Type', 'Text']).Score.transform(max)
                dftestCol = dftestCol.loc[dftestCol.Score == c_maxes]
            elif "http://healthlake.amazonaws.com/aws-cm/detect-entities/aws-cm-de-entity" in ld[0][1]:
                parseSecondLayer=ld[1][1]
                tempDict = {}
                for i in range(0, len(parseSecondLayer)):
                    if 'http://healthlake.amazonaws.com/aws-cm/detect-entities/aws-cm-de-entity-text' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Text': list(parseSecondLayer[i].values())[1]})
                        tempDict.update({'File': 'source/'+df['unique_id'][eachRow]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/detect-entities/aws-cm-de-entity-score' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Score': list(parseSecondLayer[i].values())[1]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/detect-entities/aws-cm-de-entity-type' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Type': list(parseSecondLayer[i].values())[1]})
                dftestCol = dftestCol.append(pd.DataFrame(tempDict, index=[0]), ignore_index=True)
                c_maxes = dftestCol.groupby(['Type', 'Text']).Score.transform(max)
                dftestCol = dftestCol.loc[dftestCol.Score == c_maxes]
            elif "http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity" == ld[0][1]:
                parseSecondLayer=ld[1][1]
                tempDict = {}
                for i in range(0, len(parseSecondLayer)):
                    tempDict.update({'File': 'source/'+df['unique_id'][eachRow]})
                    if 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-text' in list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Text': list(parseSecondLayer[i].values())[1]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-score' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Score': list(parseSecondLayer[i].values())[1]})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-begin-offset' == list(parseSecondLayer[i].values())[0]:
                        tempDict.update({'Type': 'RxNorm'})
                    elif 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-ConceptList' == list(parseSecondLayer[i].values())[0]:
                        additional_extension=list(parseSecondLayer[i].values())[1]
                        for j in range(0, len(additional_extension)):
                            for k in range(0, len(additional_extension[j]['extension'])):
                                if 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-Concepts-Code' == list(additional_extension[0]['extension'][k].values())[0]:
                                    Code = list(additional_extension[j]['extension'][k].values())[1]
                                elif 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-Concepts-Description' == list(additional_extension[0]['extension'][k].values())[0]:
                                    DescriptionText = list(additional_extension[j]['extension'][k].values())[1]
                                elif 'http://healthlake.amazonaws.com/aws-cm/infer-rxnorm/aws-cm-rxnorm-entity-Concepts-Score' == list(additional_extension[0]['extension'][k].values())[0]:
                                    Score = list(additional_extension[j]['extension'][k].values())[1]
                            dftestCol = dftestCol.append(pd.DataFrame({'Score': Score, 'Type': 'RxnormCode', 'Text': Code, 'File': 'source/'+df['unique_id'][eachRow]}, index=[0]), ignore_index=True)
                            dftestCol = dftestCol.append(pd.DataFrame({'Score': Score, 'Type': 'RxnormDescription', 'Text': DescriptionText, 'File': 'source/'+df['unique_id'][eachRow]}, index=[0]), ignore_index=True)
                        c_maxes = dftestCol.groupby(['Type', 'Text']).Score.transform(max)
                        dftestCol = dftestCol.loc[dftestCol.Score == c_maxes]
                dftestCol = dftestCol.append(pd.DataFrame(tempDict, index=[0]), ignore_index=True)
                c_maxes = dftestCol.groupby(['Type', 'Text']).Score.transform(max)
                dftestCol = dftestCol.loc[dftestCol.Score == c_maxes]
    dftestCol.to_csv(os.path.join(pathtosave, df['patient_id'][eachRow]+'___'+df['id'][eachRow]+'.csv'), index = False)
    boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('stdized-data/comprehend_results/csv/', df['patient_id'][eachRow]+'___'+df['id'][eachRow]+'.csv')).upload_file(df['patient_id'][eachRow]+'___'+df['id'][eachRow]+'.csv')
    os.remove(df['patient_id'][eachRow]+'___'+df['id'][eachRow]+'.csv')

In [ ]:
dftestCol